In [39]:
import subprocess

def run_command(command, verbose=True):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True)
    returncode = process.wait()
    output = process.stdout.read().decode('utf-8', 'replace').replace("\r", "").split("\n")
    if verbose: 
        for line in output:
            print (line)
    return output

In [43]:
o = run_command('D: && cd "D:\OneDrive - Universita degli Studi Roma Tre\Corsi\Big Data\Secondo Progetto\BigData-Docker" && docker-compose up -d')

redis is up-to-date
mongo is up-to-date
voltdb is up-to-date
orientdb is up-to-date
cockroach is up-to-date
memsql is up-to-date
neo4j is up-to-date
nuodb is up-to-date
postgres is up-to-date
mysql is up-to-date
cassandra is up-to-date
trafodion is up-to-date



In [44]:
o = run_command('docker exec -i voltdb echo pippo')

pippo



In [45]:
o = run_command('docker exec -i voltdb ls /shared_data')

mock_data_1000_rows.json
mock_data_1000_rows.sql
mock_data_1000_rows.tsv



In [ ]:
containers = ["mongo", "redis", "neo4j", "orientdb", "cassandra", "postgres", "mysql", "memsql", "voltdb", "cockroach"]
data_import_commands = [
    "mongoimport --db testing_db --collection customers --type csv --headerline --file /shared_data/mock_data_10000_rows.csv",
    "cat /shared_data/mock_data_10000_rows2.csv | redis-cli --pipe",
    "/var/lib/neo4j/bin/neo4j-admin import --mode=csv --database=mock.db  --nodes:Customer /shared_data/mock_data_10000_neo.csv --id-type=string",
    "/orientdb/bin/oetl.sh /shared_scripts/import_orient_10000.json",
    "cqlsh -f /shared_data/import_mock_10000.cql",
    "",
    "",
    "",
    "",
    ""
]
